In [ ]:
# Daten einlesen und vorbereiten
import pandas as pd
import plotly.graph_objects as go

# Daten aus Excel
df = pd.read_excel("01.01.2015-31.12.2024_Sonnenscheindaten_001.xlsx", sheet_name="Sheet1")

# Gruppieren & Kumulieren
grouped = df.groupby(["Jahr", "Ort_x", "Latitude_decimal", "Longitude_decimal"]).agg({"Sonnenstunden": "sum"}).reset_index()
grouped["Sonnenstunden_kumuliert"] = grouped.groupby("Ort_x")["Sonnenstunden"].cumsum()
jahre = sorted(grouped["Jahr"].unique())

# Farbstufenfunktion
def get_color(value):
    if value <= 2500: return "rgb(0, 0, 130)"
    elif value <= 5000: return "rgb(0, 0, 255)"
    elif value <= 7500: return "rgb(100, 149, 237)"
    elif value <= 10000: return "rgb(60, 179, 113)"
    elif value <= 12500: return "rgb(173, 255, 47)"
    elif value <= 15000: return "rgb(255, 255, 0)"
    elif value <= 17500: return "rgb(255, 140, 0)"
    elif value <= 20000: return "rgb(255, 69, 0)"
    else: return "rgb(139, 0, 0)"

# Frames für Animation
visual_scale = 1.0  # keine künstliche Höhendehnung
frames = []
for jahr in jahre:
    df_jahr = grouped[grouped["Jahr"] == jahr]
    traces = [
        go.Scatter3d(
            x=[row["Longitude_decimal"]]*2,
            y=[row["Latitude_decimal"]]*2,
            z=[0, row["Sonnenstunden_kumuliert"] * visual_scale],
            mode="lines",
            line=dict(color=get_color(row["Sonnenstunden_kumuliert"]), width=4),
            text=f"{row.Ort_x}<br>{int(row.Sonnenstunden_kumuliert)} Sonnenstunden",
            hoverinfo="text",
            showlegend=False
        ) for _, row in df_jahr.iterrows()
    ]
    frames.append(go.Frame(data=traces, name=str(jahr)))

# Startbild (erstes Jahr)
df_start = grouped[grouped["Jahr"] == jahre[0]]
start_traces = [
    go.Scatter3d(
        x=[row["Longitude_decimal"]]*2,
        y=[row["Latitude_decimal"]]*2,
        z=[0, row["Sonnenstunden_kumuliert"] * visual_scale],
        mode="lines",
        line=dict(color=get_color(row["Sonnenstunden_kumuliert"]), width=4),
        text=f"{row.Ort_x}<br>{int(row.Sonnenstunden_kumuliert)} Sonnenstunden",
        hoverinfo="text",
        showlegend=False
    ) for _, row in df_start.iterrows()
]

# Layout
fig = go.Figure(
    data=start_traces,
    layout=go.Layout(
        title="Sonnenstunden (2015–2024) – dynamisch mit Farbstufen",
        scene=dict(
            xaxis=dict(title="Längengrad", showgrid=True, zeroline=True, showbackground=True),
            yaxis=dict(title="Breitengrad", showgrid=True, zeroline=True, showbackground=True),
            zaxis=dict(
                title="Kumulierte Sonnenstunden",
                showgrid=False,
                zeroline=False,
                showbackground=False,
                range=[0, int(grouped["Sonnenstunden_kumuliert"].max())]
            ),
            aspectmode='manual',
            aspectratio=dict(x=2, y=1.5, z=0.6)
        ),
        sliders=[{
            "steps": [{
                "method": "animate",
                "label": str(j),
                "args": [[str(j)], {"mode": "immediate", "frame": {"duration": 500, "redraw": True}}]
            } for j in jahre]
        }],
        updatemenus=[{
            "type": "buttons",
            "buttons": [
                {"label": "Play", "method": "animate", "args": [None]},
                {"label": "Pause", "method": "animate", "args": [[None], {"mode": "immediate", "frame": {"duration": 0}}]}
            ]
        }]
    ),
    frames=frames
)

# Speichern
fig.write_html("sonnenstunden_3d_final.html")
